In [ ]:
import networkx as nx 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import optimize
import scipy
import sys
import pandas as pd
from src import SimGame as sg
from src import helperfunctions as hf
from src import isingNumba as ising

In [ ]:
def average_degree(system,control,budget):
    return abs(np.sum([system.graph.degree[i]*con for i,con in enumerate(control)])/budget)



sizes = [50,50] # sizes of blocks
probs = [[0.2, 0.05], [0.05, 0.05]]
G = nx.stochastic_block_model(sizes, probs, seed=0)

nx.draw(G)

# G.graph['partition'][1] get nodes from 2nd stochastic block


beta_crit = hf.crit_b(G)

In [ ]:

def init_allocation(budget_pos,budget_neg):
    pos_share_tight = np.random.uniform(0,1)
    neg_share_tight=np.random.uniform(0,1)

    a_pos = np.random.dirichlet(np.ones(50))*pos_share_tight*budget_pos 
    b_pos =np.random.dirichlet(np.ones(50))*(1.0-pos_share_tight)*budget_pos 

    init_pos = np.concatenate([a_pos,b_pos])

    a_neg = np.random.dirichlet(np.ones(50))*neg_share_tight*budget_neg
    b_neg =np.random.dirichlet(np.ones(50))*(1.0-neg_share_tight)*budget_neg 

    init_neg =  np.concatenate([a_neg,b_neg])

    init_alloc = np.column_stack([init_pos,init_neg]).T
    return init_alloc


def average_degree(system,control,budget):
    return np.sum([system.graph.degree[i]*con for i,con in enumerate(control)])/budget



In [ ]:
X = dict.fromkeys([0,1],None)
Y= dict.fromkeys([0,1],None)

In [ ]:
b = np.zeros(len(G.nodes.keys()))
budget_pos=20.0
budget_neg=20.0


system = ising.mf_ising_system(G,b,iim_iter=10000,iim_tol_fac=1e-8)


beta=beta_crit*10.0

iters=5
pos_degrees=[]
neg_degrees=[]

for i in range(iters):
    init_external=init_allocation(budget_pos,budget_neg)
    control_pos,control_neg,mag=system.MF_IIM(budget_pos,budget_neg,beta,init_alloc=init_external)
    if system.converged==False:
        continue
    control_pos_history = system.control_field_history_pos
    control_neg_history = system.control_field_history_neg
    temp_pos=[]
    temp_neg=[]
    for idx in range(0,control_neg_history.shape[0]):
        pos_degree = average_degree(system,control_pos_history[idx],budget_pos)
        neg_degree = average_degree(system,control_neg_history[idx],budget_neg)
        temp_pos.append(pos_degree)
        temp_neg.append(neg_degree)
    
    pos_degrees.append(temp_pos)
    neg_degrees.append(temp_neg)

In [ ]:
if beta==beta_crit/10.0:
    X[0]=pos_degrees
    Y[0]=neg_degrees
if beta==beta_crit*10.0:
    X[1]=pos_degrees
    Y[1]=neg_degrees
       


In [ ]:
s=np.array(system.pos_gradient_history)

v=np.array(system.neg_gradient_history)


plt.plot(np.sum(s,axis=1))
plt.plot(np.sum(v,axis=1))

In [ ]:
import matplotlib

n=1

matplotlib.rcParams.update({'font.size': 18,'figure.figsize':(12,10),'axes.labelsize': 'large',
                           'axes.titlesize':20})
s=23
fig,ax=plt.subplots(figsize=(15,7),ncols=2)

for idx,run in enumerate(X[0]):
    
    ax[0].plot(run[::n],Y[0][idx][::n],label='Convergence paths',c='blue',linestyle='dashed',alpha=0.2)
    ax[0].set_xlabel('Average degree - positive agent')
    ax[0].set_ylabel('Average degree - negative agent')
    ax[0].scatter(run[0],Y[0][idx][0],c='green',label='start points',s=50)
    ax[0].scatter(run[-1],Y[0][idx][-1],c='red',label='final points')

    hand, labl = ax[0].get_legend_handles_labels()
    handout=[]
    lablout=[]
    for h,l in zip(hand,labl):
        if l not in lablout:
            lablout.append(l)
            handout.append(h)
    ax[0].legend(handout, lablout,loc='best')
    ax[0].set_title(r'$\beta$={}'.format(np.round(beta_crit/10.0,3)),fontsize=s)

for idx,run in enumerate(X[1]):
    
    ax[1].plot(run[::n],Y[1][idx][::n],label='Convergence paths',c='blue',linestyle='dashed',alpha=0.2)
    ax[1].set_xlabel('Average degree - positive agent')
    ax[1].set_ylabel('Average degree - negative agent')
    ax[1].scatter(run[0],Y[1][idx][0],c='green',label='start points',s=50)
    ax[1].scatter(run[-1],Y[1][idx][-1],c='red',label='final points')

    hand, labl = ax[1].get_legend_handles_labels()
    handout=[]
    lablout=[]
    for h,l in zip(hand,labl):
        if l not in lablout:
            lablout.append(l)
            handout.append(h)
    ax[1].legend(handout, lablout,loc='best')
    ax[1].set_title(r'$\beta$={}'.format(np.round(beta_crit*10.0,3)),fontsize=s)



In [ ]:
budget_pos=20.0
b = np.zeros(len(G.nodes.keys()))

budget_neg=20.0

iters=10
beta_min = 0.5*beta_crit 
beta_max = 3*beta_crit 
iters=10
beta_list = np.logspace(np.log10(beta_min),np.log10(beta_max),num=20,endpoint=True,base=10)
control_history_pos=np.zeros((iters,len(beta_list),2))
control_history_neg=np.zeros((iters,len(beta_list),2))
mag_history = np.zeros((len(beta_list),100))

for it in range(iters):
    print('iteration ',it)
    for i,beta in enumerate(beta_list):
        system =ising.mf_ising_system(G,b,iim_iter=10000,iim_tol_fac=1e-5)
        control_pos,control_neg,final_mag=system.MF_IIM(budget_pos,budget_neg,beta,progress=False)

        block1,block2 = np.sum(control_pos[:50]),np.sum(control_pos[50:])
        control_history_pos[it,i]=np.array([block1,block2])

        block1,block2 = np.sum(control_neg[:50]),np.sum(control_neg[50:])
        control_history_neg[it,i]=np.array([block1,block2])

        mag_history[i]=final_mag


print('FINISHED')

In [ ]:
control_pos_mean = np.mean(control_history_pos,axis=0)
control_neg_mean = np.mean(control_history_neg,axis=0)

control_pos_std = np.std(control_history_pos,axis=0)
control_neg_std = np.std(control_history_neg,axis=0)



In [ ]:
import matplotlib


matplotlib.rcParams.update({'font.size': 14,'figure.figsize':(12,5),'axes.labelsize': 'large',
                           'axes.titlesize':18})

fig,ax = plt.subplots(ncols=2)

ax[0].plot(beta_list,control_pos_mean[:,0],c='orange',label='Block 1',linestyle='dashed',linewidth=3)
ax[0].fill_between(beta_list,control_pos_mean[:,0]+control_pos_std[:,0],control_pos_mean[:,0]-control_pos_std[:,0]
                   ,**{'color':'orange','alpha':0.5})

ax[0].plot(beta_list,control_pos_mean[:,1],c='green',label='Block 2',linestyle='dashed',linewidth=3)
ax[0].fill_between(beta_list,control_pos_mean[:,1]+control_pos_std[:,1],control_pos_mean[:,1]-control_pos_std[:,1]
                   ,**{'color':'green','alpha':0.5})


ax[0].set_xscale(value='log')

ax[0].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax[0].set_ylabel('Budget spent')
ax[0].vlines(beta_crit,ymin=0,ymax=20,linestyles='dashed',color='black',label=r'$\beta_{crit}$')
ax[0].set_xlabel(r'Interaction Strenght $\beta$')
ax[0].set_title('Positive agent')
ax[0].legend()


ax[1].plot(beta_list,control_neg_mean[:,0],c='orange',label='Block 1',linestyle='dashed',linewidth=3)
ax[1].fill_between(beta_list,control_neg_mean[:,0]+control_neg_std[:,0],control_neg_mean[:,0]-control_neg_std[:,0]
                   ,**{'color':'orange','alpha':0.5})


ax[1].plot(beta_list,control_neg_mean[:,1],c='green',label='Block 2',linestyle='dashed',linewidth=3)
ax[1].fill_between(beta_list,control_neg_mean[:,1]+control_neg_std[:,1],control_neg_mean[:,1]-control_neg_std[:,1]
                   ,**{'color':'green','alpha':0.5})

ax[1].set_xscale(value='log')

ax[1].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax[1].set_ylabel('Budget spent')
ax[1].vlines(beta_crit,ymin=0,ymax=20,linestyles='dashed',color='black',label=r'$\beta_{crit}$')
ax[1].set_xlabel(r'Interaction Strenght $\beta$')
ax[1].set_title('Negative agent')
ax[1].legend()


fig.suptitle(r'Budget ratio: $B_{pos}$:$B_{neg}$=1:1',fontsize=20)
plt.tight_layout()
plt.show()


In [ ]:
budget_pos=5.0
budget_neg=10.0


lr_1 = lambda x,maxiter : 5*np.exp(-x/(0.9*maxiter))


lr_2 = lambda x,maxiter : np.exp(-x/(0.9*maxiter))


beta=0.2
b = np.zeros(len(G.nodes.keys()))
iters=10


params = {'optimiser_type':'adam','lr_1':lr_1,'lr_2':lr_2,'beta1':0.9,'beta2':0.999,'eps':10.0}

controlArr = []
for i in range(iters):
    system = sg.mf_ising_system(G,b,iim_iter=5000,fixed_point_iter=int(5*1e5),iim_tol_fac=1e-3,**params)
    init_external=init_allocation(budget_pos,budget_neg)
    control_pos,control_neg,mag=system.MF_IIM(budget_pos,budget_neg,beta,init_alloc=init_external)
    controlArr.append(np.array(system.mag_delta_history))


In [ ]:
x=np.sum(np.array(system.mag_delta_history),axis=1)

y = np.std(np.array(system.mag_delta_history),axis=1)


In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 20,'figure.figsize':(12,10),'axes.labelsize': 'large',
                           'axes.titlesize':18})


fig,ax=plt.subplots()

xarray=np.arange(0,len(x),1)
ax.plot(x,label='Total Magnetisation')
ax.fill_between(xarray,x+y,x-y,**{'color':'blue','alpha':0.5})


ax.legend()

        
pos_string = '$Budget_{pos}$'
neg_string = '$Budget_{neg}$'
ax.set_title(r'Overall nodes magnetisation'+'\n' +'for {}={} {}={}'.format(pos_string,budget_pos,neg_string,budget_neg),fontsize=22)
ax.set_xlabel('Iterations')
ax.set_ylabel('Magnetisation sum')